In [1]:
import json
import re
import urllib.request

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
ROCHADE_URL = "https://rochadeeuropa.com/lichess-turniere-beendet/"
BUNDESLIGA_REGEXP = r"|".join(["(1. DE-Quarantäne Team Battle)",
                               "([0-9]+\. ?DE[ -]Quarantäne Teams 1-10)",
                               "(5. Quarantäne-Liga Teams 1-10)",
                               "([0-9]+\. ?Quarantäne-Bundesliga)"
                              ])


In [3]:
def get_bundesliga_tournaments():
    """
    scrape rochadeeuropa.com in order to find lichess quarantaine bundesliga matches.
    Rochade URL and regex to determine which tournament was actually a bundesliga tournaments are 
    taken from configurable global variables
    """
    # scrape rochade finished lichess tournaments
    response = urllib.request.urlopen(ROCHADE_URL)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    # parse table
    data = []
    table = soup.find('table', attrs={'class':'tablepress tablepress-id-3'})
    table_body = table.find('tbody')
    
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        data.append([ele.text.strip() for ele in cols])
    # filter for bundesliga
    buli_tournaments = [el for el in data if re.match(BUNDESLIGA_REGEXP, el[1])]
    return buli_tournaments

In [4]:
def get_individual_results(tournament_id):
    """
    For individual results for a given tournament we use lichess API
    """
    api_url = f"https://lichess.org/api/tournament/{tournament_id}/results"
    api_response = urllib.request.urlopen(api_url)
    player_results = api_response.read()
    return pd.DataFrame([json.loads(pl) for pl in player_results.splitlines()])

In [5]:
def get_team_results(tournament_url):
    """
    Unfortunately the API does not provide team results, hence we need to go the pedestrian way and 
    scrape it by hand
    """
    tournament_response = urllib.request.urlopen(tournament_url)
    tournament_html = tournament_response.read()
    tournament_soup = BeautifulSoup(tournament_html, 'html.parser')
    # there is a string before the json that needs to be deleted.
    # This seems a little clumsy and error prone but there is no other way when parsing manually
    str_to_delete = "lichess.tournament="
    tournament_strings = [script.text for script in tournament_soup.find_all("script") 
                          if str_to_delete in script.text]
    tournament_strings.sort(key=lambda s: len(s))
    tournament_string = tournament_strings[-1]
    tournament_json = json.loads(tournament_string[tournament_string.find(str_to_delete)+len(str_to_delete):])
    team_name_dict = tournament_json["data"]["teamBattle"]["teams"]
    relevant_keys = ["rank", "id", "score"]
    team_results = [{k: v for k, v in team_dict.items() if k in relevant_keys} for team_dict in 
                    tournament_json["data"]["teamStanding"]]
    teams_df = pd.DataFrame(team_results)
    teams_df["Team"] = teams_df["id"].map(team_name_dict)
    return teams_df

In [6]:
team_df = pd.DataFrame()
individual_df = pd.DataFrame()

buli_tournaments = get_bundesliga_tournaments()
for tournament in buli_tournaments:
    print(f"Downloading data of tournament {tournament[1]} on {tournament[0]}")
    team_df = team_df.append(get_team_results(tournament[4]))
    individual_df = individual_df.append(get_individual_results(tournament[4].split("/")[-1]))
    
team_df["count"] = 1
team_df["champion"] = (team_df["rank"] == 1).astype(int)
individual_df["count"] = 1
individual_df["champion"] = (individual_df["rank"] == 1).astype(int)

In [7]:
all_time_teams = team_df.groupby("Team", as_index=False).agg(
    {"score": "sum", "count": "count", "champion": "sum", "rank": "mean"}).sort_values("score", ascending=False)
all_time_teams.columns = ["Team", "Gesamtpunkte", "Teilnahmen", "Meisterschaften", "Durchschnittsplatzierung"]
all_time_teams.index = range(1, len(all_time_teams) + 1)
all_time_indiv = individual_df.groupby("username", as_index=False).agg(
    {"score": "sum", "count": "count", "champion": "sum", "performance": "mean", "rank": "mean", }).sort_values("score", ascending=False)
all_time_indiv.columns = ["Name", "Gesamtpunkte", "Teilnahmen", "Turniersiege", "Durchschnittsperformance", "Durchschnittsplatzierung"]
all_time_indiv.index = range(1, len(all_time_indiv) + 1)
all_time_indiv["Durchschnittsscore"] = all_time_indiv["Gesamtpunkte"] / all_time_indiv["Teilnahmen"]

In [8]:
all_time_teams.reset_index().rename(columns={"index": "Rang"})

,Rang,Team,Gesamtpunkte,Teilnahmen,Meisterschaften,Durchschnittsplatzierung
0,1,Bierstube MUC,3975,15,3,2.533333
1,2,HSK Lister Turm,3035,11,4,3.636364
2,3,Oxfordgang,2692,12,0,5.750000
3,4,KSK Dr. Lasker 1861 e.V.,2365,9,0,5.222222
4,5,SK Landau 1908 e.V.,2254,8,0,5.875000
5,6,TSV Schöniach,2226,9,0,5.222222
6,7,Die Nordlichter,1859,9,3,5.111111
7,8,Hessische Schachjugend,1616,6,1,6.333333
8,9,Kingdom,1287,4,0,3.750000
9,10,Wiener Kandidaten und Fake Meister,1203,5,0,5.800000


In [9]:
all_time_indiv.round({"Durchschnittsplatzierung": 1, "Durchschnittsperformance": 1, "Durchschnittsscore": 1})

,Name,Gesamtpunkte,Teilnahmen,Turniersiege,Durchschnittsperformance,Durchschnittsplatzierung,Durchschnittsscore
1,SomeMightSay,821,15,4,2570.9,4.1,54.7
2,psammenitos,508,10,4,2577.1,16.7,50.8
3,Satscho,487,15,0,2339.6,37.3,32.5
4,Kabeljaukrieger,481,14,0,2391.2,29.5,34.4
5,Chillkroete77,470,12,1,2356.2,16.5,39.2
6,P1W4,418,15,0,2177.1,55.5,27.9
7,Eldrail,415,11,0,2523.5,26.7,37.7
8,fluffy_cactus,408,15,0,2232.7,58.2,27.2
9,Veez,391,12,0,2349.1,33.0,32.6
10,Blend99,388,8,0,2506.6,8.0,48.5


In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def connect_to_spreadsheet():
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']

    credentials = ServiceAccountCredentials.from_json_keyfile_name('/home/paul/Downloads/dummyproject-273213-6c2b9c180973.json', scope)

    gc = gspread.authorize(credentials)
    return gc

In [11]:
import gspread_dataframe as gd

def write_to_spreadsheet(df, sheet_connection):
    ws = sheet_connection.open("Ewige Quarantäne-Bundesligatabelle").worksheet("Sheet1")
    gd.set_with_dataframe(ws, df)

In [12]:
df_to_write = all_time_teams.reset_index().rename(columns={"index": "Rang"}).round({"Durchschnittsplatzierung": 1})

In [13]:
conn = connect_to_spreadsheet()

In [14]:
write_to_spreadsheet(df_to_write, conn)